
# Varios métodos de predicción por departamento con optimización de parámetros

*@autor: Jesús Manuel Puentes Gutiérrez*

*@version: 5*

*Universidad de Alcalá*
______________________________________________
 

Se generan varios algortimos predictivos para poder predecir el número estimados de días que permanecerá ingresado un paciente en un hospital en función de cada departamento hospitalario.

Se utilizan diferentes algoritmos de Clasificación.



______________________________________________


### Librerías comunes

Se deben cargar las siguientes librerías que son comunes a todos los algoritmos

In [1]:
import csv
import random
import math
import operator

import numpy as np
from matplotlib import pyplot as plt

from time import time
from scipy.stats import randint as sp_randint
from sklearn import neighbors

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


### Datos

Generamos la variable que contiene el archivo con los datos del departamento a analizar



Nombre del departamento: **OBSTETRICS CON PATRÓN**


In [210]:
dat1 = '/Users/shuli/Desktop/Universidad de Alcalá/DOCTORADO/DataSet/Data set CAR PSQ H12O/'
dat2 = 'Dataset_por_departamentos_sin_cabecera/dep_sincab_obsEmbPT.csv'
datos = dat1 + dat2

dep = 'OBSTETRICS CON PATRÓN'

#### Función loadDataset
Con esta función seleccionaremos los datos de entrada de forma aleatoria

In [211]:
def loadDataset_XX(filename, split, trainingSet=[] , testSet=[]):
	with open(filename) as csvfile:
	    lines = csv.reader(csvfile)
	    dataset = list(lines)
	    for x in range(len(dataset)-1):
	        for y in range(11):
	            dataset[x][y] = float(dataset[x][y])
	        if random.random() < split:
	            trainingSet.append(dataset[x])
	        else:
	            testSet.append(dataset[x])

#### Función getAccuracy
Con esta función obtenemos la precisión con la que se realizan las predicciones mediante los datos de prueba seleccionados

In [212]:
def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

#### Función getAccuracy_XX
Con esta función obtenemos la precisión con la que se realizan las predicciones para valores tipo float, mediante los datos de prueba seleccionados

In [213]:
def getAccuracy_XX(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

#### Función getAccuracy2_XX

Con esta función obtenemos la precisión con la que se realizan las predicciones para valores tipo float, mediante los datos de prueba seleccionados, teniendo en cuenta también el porcentaje de acierto cuando no es exacto el resultado.

In [214]:
def getAccuracy2_XX(testSet, predictions):
    percent = 0.0
    for x in range(len(testSet)):
        if testSet[x] == predictions[x]:
            percent = (percent + 100.0) / 2.0
        else:
            if testSet[x] == 0.0:
                parcial = (predictions[x]*100.0)
            else:
                parcial = (predictions[x]*100.0) / testSet[x]
            if parcial > 100.0:
                percent2 = 100.0 - (parcial - 100.0)
                percent = (percent + percent2) / 2.0
            else:
                percent = (percent + parcial) / 2.0
    return percent

#### Función getError_XX

Con esta función obtenemos la precisión con la que se realizan las predicciones para valores tipo float, mediante los datos de prueba seleccionados, teniendo en cuenta también el porcentaje de acierto cuando no es exacto el resultado. Por lo tanto se calcula el porcentaje de error al predecir los resultados (lo que se aleja de la predicción exacta).

Esta forma de calcular el porcentaje de acierto nos permite saber con más exactitud dicho porcentaje teniendo en cuenta lo que se acerca el resultado predicho al resultado real, no solo acertando exactamente el número.

In [215]:
def getError_XX(testSet, predictions):
    percentError = 0.0
    for x in range(len(testSet)):
        if testSet[x] == predictions[x]:
            percentError = (percentError + 0.0) / 2.0
        else:
            if testSet[x] == 0.0:
                parcial = (predictions[x]*100.0)
            else:
                parcial = np.abs(100 - ((predictions[x]*100.0) / testSet[x]))
                percentError = (percentError + parcial) / 2.0
    return percentError

#### Función getError2_XX

Función que calcula el error de forma más exacta.

In [216]:
def getError2_XX(testSet, predictions):
    percentError = []
    for x in range(len(testSet)):
        if predictions[x] < 0.0:
            predict = 0.0
        else:
            predict = predictions[x]
        if testSet[x] == predictions[x]:
            Error = 0.0
            percentError.append(Error)
        else:
            if testSet[x] == 0.0:
                parcial = np.abs(testSet[x]-predict)*100.0
                percentError.append(parcial)
            else:
                parcial = (np.abs(testSet[x]-predict)*100.0) / testSet[x]
                percentError.append(parcial)
    return sum(percentError)/len(percentError)


### Programa principal

Preparamos los datos y los cargamos de nuestro dataset preparado en un archivo tipo *csv*.

Seleccionamos un **67%** de los datos para entrenamiento y el resto para prueba

In [217]:
trainingSet=[]
testSet=[]
split = 0.67

# Parte específica del algoritmo KNN
n_neighbors = 15
weights = 'distance'

# En Windows
# loadDataset(r'C:\Users\Chus\Google Drive\DOCTORADO\DataSet\Data set CAR PSQ H12O\cmbd10.csv',
#              split, trainingSet, testSet)

# En Mac
loadDataset_XX(datos, split, trainingSet, testSet)

Mostramos como se han distribuido los datos de entrenamiento y los de prueba

In [218]:
print ('Train set: ' + repr(len(trainingSet)))
print ('Test set: ' + repr(len(testSet)))

# Lo guardamos en variables para cuadro resumen
entrenamiento = repr(len(trainingSet))
pruebas       = repr(len(testSet))

Train set: 218
Test set: 103


Se eligen y preparan los datos que se van a usar como entrenamiento:

- Se selecciona como parte **X** (características del dataset) todas las variables menos la última
- Se selecciona como parte **y** (clases del dataset) la última variable del dataset, que es la variable resultado

In [219]:
X = []
y = []

for x in range(len(trainingSet)):
    X.append(trainingSet[x][:-1])
    
for x in range(len(trainingSet)):
    y.append(trainingSet[x][-1])

# Caso todas edades son 0 (neonatos)
#m = 0.0

#for n in range(len(X)):
#    if np.max(np.abs(X[n][7])) > m:
#        m = np.max(np.abs(X[n][7]))

#if m == 0.0:
#    for n in range(len(X)):
#        X[n][7] = 0.0000001

Se eligen y preparan los datos que se van a usar como pruebas:

- Se selecciona como parte **X_test** (características del dataset) todas las variables menos la última
- Se selecciona como parte **y_test** (clases del dataset) la última variable del dataset. Esta variable contendrá los valores reales que deberán compararse con la variable predicha para ver su porcentaje de acierto

In [220]:
X_test = []
y_test = []

for x in range(len(testSet)):
    X_test.append(testSet[x][:-1])

for x in range(len(testSet)):
    y_test.append(testSet[x][-1])

# Caso todas edades son 0 (neonatos)
#m = 0.0

#for n in range(len(X_test)):
#    if np.max(np.abs(X_test[n][7])) > m:
#        m = np.max(np.abs(X_test[n][7]))

#if m == 0.0:
#    for n in range(len(X_test)):
#        X_test[n][7] = 0.0000001

## Algoritmo KNN (Kneighborsclassifier) con K vecinos más cercanos - Sklearn


### Librerías específicas del algoritmo

Se debe cargar la siguiente librería adicional para trabajar con el nuevo algoritmo

In [221]:
from sklearn.neighbors import KNeighborsClassifier


### Programa principal del algoritmo seleccionado

Los datos necesarios del departamento se encuentran ya cargados dentro de las listas **testSet** y **trainingSet**. Se ha seleccionado un **67%** de los datos para entrenamiento y el resto para prueba. La distribución de los registros se puede ver al principio de este documento, dentro del primer algoritmo utilizado (*KNN*).

Los datos que se van a utilizar para el entrenamiento (**X** e **y**) y los que se van a utilizar para las pruebas (**X_test** e **y_test**) se reutilizan ya que no se han modificado.


#### Algoritmo

A continuación seleccionamos los mejores parámetros a utilizar para el dataset elegido.


In [222]:
# Definimos las opciones a elegir
weight_options = ['uniform', 'distance']

# Elegimos el número de vecinos mas óptimo
k_range = list(range(1, 31))

# Definimos el grupo de opciones
param_grid = dict(n_neighbors=k_range, weights=weight_options)

knn = KNeighborsClassifier()

# Probamos todas las opciones
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')

grid.fit(X, y)

/Users/shuli/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

Mostramos la mejor opción encontrada

In [223]:
grid.best_estimator_


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='distance')

Elegimos el algoritmo a utilizar y lo entrenamos

In [224]:
clf = grid.best_estimator_

clf.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='distance')

A continuación se predice el resultado con el algoritmo que hemos entrenado

In [225]:
y_predicted = []

y_predicted = clf.predict(X_test)

### Resultados


Se comparan los resultados reales (*y_test*) con los que hemos predicho y se muestra la precisión real del algoritmo 

In [226]:
accuracyKNN = getAccuracy_XX(y_test, y_predicted)
accuracy2KNN = getAccuracy2_XX(y_test, y_predicted)
errorKNN = getError_XX(y_test, y_predicted)
error2KNN = getError2_XX(y_test, y_predicted)
accuracy3KNN = 100 - error2KNN

print('Accuracy (aciertos)        : ' + repr(accuracyKNN) + '%')
print('Accuracy2                  : ' + repr(accuracy2KNN) + '%')
print('Accuracy3 (dif. con Error2): ' + repr(accuracy3KNN) + '%')
print('Error                      : ' + repr(errorKNN) + '%')
print('Error2                     : ' + repr(error2KNN) + '%')

Accuracy (aciertos)        : 29.126213592233007%
Accuracy2                  : 64.076781523738788%
Accuracy3 (dif. con Error2): 62.384805054707947%
Error                      : 35.923218476261205%
Error2                     : 37.615194945292053%


## SVM ( SUPPORT VECTOR MACHINE ) CON KERNEL RBF


### Librerías específicas del algoritmo

Se debe cargar la siguiente librería adicional para trabajar con el nuevo algoritmo

In [227]:
from sklearn.svm import SVC


### Programa principal del algoritmo seleccionado

Los datos necesarios del departamento se encuentran ya cargados dentro de las listas **testSet** y **trainingSet**. Se ha seleccionado un **67%** de los datos para entrenamiento y el resto para prueba. La distribución de los registros se puede ver al principio de este documento, dentro del primer algoritmo utilizado (*KNN*).

Los datos que se van a utilizar para el entrenamiento (**X** e **y**) y los que se van a utilizar para las pruebas (**X_test** e **y_test**) se reutilizan ya que no se han modificado.


#### Algoritmo

A continuación seleccionamos los mejores parámetros a utilizar para el dataset elegido.


In [228]:
# Seleccionamos rangos de valores a optimizar
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]

param_grid = {'C': Cs, 'gamma' : gammas}

# Definimos el grupo de opciones
grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=5)

grid_search.fit(X,y)


/Users/shuli/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

Mostramos la mejor opción encontrada

In [229]:
grid_search.best_estimator_

SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Elegimos el algoritmo a utilizar y lo entrenamos

In [230]:
rbf_svc = grid_search.best_estimator_

rbf_svc.fit(X, y)

SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

A continuación se predice el resultado con el algoritmo que hemos entrenado

In [231]:
y_predicted = []

y_predicted = rbf_svc.predict(X_test)

### Resultados


Se comparan los resultados reales (*y_test*) con los que hemos predicho y se muestra la precisión real del algoritmo 

In [232]:
accuracyRBF = getAccuracy_XX(y_test, y_predicted)
accuracy2RBF = getAccuracy2_XX(y_test, y_predicted)
errorRBF = getError_XX(y_test, y_predicted)
error2RBF = getError2_XX(y_test, y_predicted)
accuracy3RBF = 100 - error2RBF

print('Accuracy (aciertos)        : ' + repr(accuracyRBF) + '%')
print('Accuracy2                  : ' + repr(accuracy2RBF) + '%')
print('Accuracy3 (dif. con Error2): ' + repr(accuracy3RBF) + '%')
print('Error                      : ' + repr(errorRBF) + '%')
print('Error2                     : ' + repr(error2RBF) + '%')

Accuracy (aciertos)        : 31.06796116504854%
Accuracy2                  : 50.469134772998828%
Accuracy3 (dif. con Error2): 71.431653567575893%
Error                      : 49.530865227001172%
Error2                     : 28.568346432424111%



## RANDOM FOREST




### Librerías específicas del algoritmo

Se debe cargar la siguiente librería adicional para trabajar el nuevo algoritmo



In [233]:
from sklearn.ensemble import RandomForestClassifier

#### Función específica report

Función utilizada para mostrar los mejores resultados a nivel eficiencia en el algoritmo **Random Forest**.

In [234]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


### Programa principal del algoritmo seleccionado

Los datos necesarios del departamento se encuentran ya cargados dentro de las listas **testSet** y **trainingSet**. Se ha seleccionado un **67%** de los datos para entrenamiento y el resto para prueba. La distribución de los registros se puede ver al principio de este documento, dentro del primer algoritmo utilizado (*KNN*).

Los datos que se van a utilizar para el entrenamiento (**X** e **y**) y los que se van a utilizar para las pruebas (**X_test** e **y_test**) se reutilizan ya que no se han modificado.


#### Algoritmo

A continuación seleccionamos los mejores parámetros a utilizar para el dataset elegido.


In [235]:
rf = RandomForestClassifier(n_estimators=20)

# especificamos los parámetros y distribuciones desde las que muestrear
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# ejecutamos una búsqueda aleatoria
n_iter_search = 20
random_search = RandomizedSearchCV(rf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5)


start = time()
random_search.fit(X, y)
print("RandomizedSearchCV duró %.2f segundos para %d ajustes"
      " del parámtro candidato." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5)
start = time()
grid_search.fit(X, y)

print("GridSearchCV duró %.2f segundos para %d ajustes del parámetro candidato."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

#rf.fit(X, y)

/Users/shuli/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


RandomizedSearchCV duró 4.51 segundos para 20 ajustes del parámtro candidato.
Model with rank: 1
Mean validation score: 0.472 (std: 0.048)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 3, 'max_features': 7, 'min_samples_split': 8}

Model with rank: 2
Mean validation score: 0.463 (std: 0.051)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 3, 'max_features': 9, 'min_samples_split': 10}

Model with rank: 3
Mean validation score: 0.459 (std: 0.059)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 3, 'max_features': 2, 'min_samples_split': 2}



/Users/shuli/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV duró 15.98 segundos para 72 ajustes del parámetro candidato.
Model with rank: 1
Mean validation score: 0.491 (std: 0.052)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 3, 'max_features': 10, 'min_samples_split': 10}

Model with rank: 2
Mean validation score: 0.486 (std: 0.038)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 3, 'max_features': 1, 'min_samples_split': 10}

Model with rank: 3
Mean validation score: 0.482 (std: 0.079)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': 1, 'min_samples_split': 10}

Model with rank: 3
Mean validation score: 0.482 (std: 0.055)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 3, 'max_features': 3, 'min_samples_split': 3}



#### Algoritmo

Elegimos el algoritmo a utilizar con los parámetros encontrados y lo entrenamos.

In [236]:
rf = random_search.best_estimator_

#rf = RandomForestClassifier(n_estimators=20, bootstrap='False', criterion='gini', max_depth=3, max_features=1, min_samples_split=10)

rf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features=7, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

A continuación se predice el resultado con el algoritmo que hemos entrenado

In [237]:
y_predicted = []

y_predicted = rf.predict(X_test)


### Resultados


Se comparan los resultados reales (*y_test*) con los que hemos predicho y se muestra la precisión real del algoritmo


In [238]:
accuracyRF = getAccuracy_XX(y_test, y_predicted)
accuracy2RF = getAccuracy2_XX(y_test, y_predicted)
errorRF = getError_XX(y_test, y_predicted)
error2RF = getError2_XX(y_test, y_predicted)
accuracy3RF = 100 - error2RF

print('Accuracy (aciertos)        : ' + repr(accuracyRF) + '%')
print('Accuracy2                  : ' + repr(accuracy2RF) + '%')
print('Accuracy3 (dif. con Error2): ' + repr(accuracy3RF) + '%')
print('Error                      : ' + repr(errorRF) + '%')
print('Error2                     : ' + repr(error2RF) + '%')


Accuracy (aciertos)        : 40.77669902912621%
Accuracy2                  : 73.067283229261307%
Accuracy3 (dif. con Error2): 78.792572044999233%
Error                      : 26.932716770738701%
Error2                     : 21.207427955000764%



## ÁRBOLES DE DECISIÓN


### Librerías específicas del algoritmo

Se deben cargar las siguientes librerías adicionales para trabajar el nuevo algoritmo



In [239]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

### Programa principal del algoritmo seleccionado

Los datos necesarios del departamento se encuentran ya cargados dentro de las listas **testSet** y **trainingSet**. Se ha seleccionado un **67%** de los datos para entrenamiento y el resto para prueba. La distribución de los registros se puede ver al principio de este documento, dentro del primer algoritmo utilizado (*KNN*).

Los datos que se van a utilizar para el entrenamiento (**X** e **y**) y los que se van a utilizar para las pruebas (**X_test** e **y_test**) se reutilizan ya que no se han modificado.


#### Algoritmo

A continuación seleccionamos los mejores parámetros a utilizar para el dataset elegido.

In [240]:
# Se seleccionarán los mejores parámetros
parameters={'criterion': ['gini','entropy'], 'min_samples_split' : range(2,500,10),'max_depth': range(1,120,2),
            'min_samples_leaf': [1, 5, 10], 'max_leaf_nodes': [None, 5, 10, 20]}

clf_tree=DecisionTreeClassifier()

# buscamos la mejor opción
clf=GridSearchCV(clf_tree,parameters)

clf.fit(X,y)


/Users/shuli/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'criterion': ['gini', 'entropy'], 'min_samples_split': range(2, 500, 10), 'max_depth': range(1, 120, 2), 'min_samples_leaf': [1, 5, 10], 'max_leaf_nodes': [None, 5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

Mostramos la mejor opción encontrada

In [241]:
clf.best_estimator_


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=22,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

#### Algoritmo

Elegimos el algoritmo a utilizar con los parámetros encontrados y lo entrenamos.

In [242]:
ad = clf.best_estimator_

ad.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=22,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


Verificamos la precisión del modelo


In [243]:
print("Precisión del modelo: {0: .2f}".format((y_test == ad.predict(X_test)).mean()))

Precisión del modelo:  0.41


A continuación se predice el resultado con el algoritmo que hemos entrenado

In [244]:
y_predicted = []

y_predicted = ad.predict(X_test)

### Resultados


Se comparan los resultados reales (*y_test*) con los que hemos predicho y se muestra la precisión real del algoritmo

In [245]:
accuracyAD = getAccuracy_XX(y_test, y_predicted)
accuracy2AD = getAccuracy2_XX(y_test, y_predicted)
errorAD = getError_XX(y_test, y_predicted)
error2AD = getError2_XX(y_test, y_predicted)
accuracy3AD = 100 - error2AD

print('Accuracy (aciertos)        : ' + repr(accuracyAD) + '%')
print('Accuracy2                  : ' + repr(accuracy2AD) + '%')
print('Accuracy3 (dif. con Error2): ' + repr(accuracy3AD) + '%')
print('Error                      : ' + repr(errorAD) + '%')
print('Error2                     : ' + repr(error2AD) + '%')

Accuracy (aciertos)        : 40.77669902912621%
Accuracy2                  : 85.567019941214909%
Accuracy3 (dif. con Error2): 78.378794883649263%
Error                      : 14.432980058785088%
Error2                     : 21.621205116350737%


## Cuadro Resumen

In [246]:
p = '.'
c = ','

print ('_______________________')
print ()
print ('Nombre del Departamento:', dep)
print ('_______________________')
print ()
print ('NÚMERO DE CASOS PARA ENTRENAMIENTO (67%) Y PARA PRUEBAS (33%):')
print ()
print ('   Entrenamiento:', entrenamiento)
print ('   Pruebas      :', pruebas)
print ()
print ('_______________________')
print ()
print ('PORCENTAJES DE ACIERTO EXACTOS:')
print ()
print ('KNN con K vecinos más cercanos         :  ' + repr(accuracyKNN).replace(p,c) + ' %')
#print ('SVM con Kernel polinomial Grado 4      :  ' + repr(accuracyPOL).replace(p,c) + ' %')
print ('SVM con Kernel RBF                     :  ' + repr(accuracyRBF).replace(p,c) + ' %')
#print ('SVM - SVC No Lineal                    :  ' + repr(accuracyNL).replace(p,c) + ' %')
#print ('SVM - SVC Lineal                       :  ' + repr(accuracyLN).replace(p,c) + ' %')
#print ('SVM - SVC Naïve Bayes                  :  ' + repr(accuracyNB).replace(p,c) + ' %')
print ('Árboles de Decisión                    :  ' + repr(accuracyAD).replace(p,c) + ' %')
print ('Random Forest                          :  ' + repr(accuracyRF).replace(p,c) + ' %')
#print ('Redes Neuronales (Pybrain)             :  ' + repr(100 - errorRN).replace(p,c) + ' %')
print ()
#print ('_______________________')
#print ()
#print ('PORCENTAJES DE ACIERTO2 (con acercamiento):')
#print ()
#print ('KNN con K vecinos más cercanos (K = 15):  ' + repr(accuracy2KNN).replace(p,c) + ' %')
#print ('SVM con Kernel polinomial Grado 4      :  ' + repr(accuracy2POL).replace(p,c) + ' %')
#print ('SVM con Kernel RBF                     :  ' + repr(accuracy2RBF).replace(p,c) + ' %')
#print ('SVM - SVC No Lineal                    :  ' + repr(accuracy2NL).replace(p,c) + ' %')
#print ('SVM - SVC Lineal                       :  ' + repr(accuracy2LN).replace(p,c) + ' %')
#print ('SVM - SVC Naïve Bayes                  :  ' + repr(accuracy2NB).replace(p,c) + ' %')
#print ('Árboles de Decisión                    :  ' + repr(accuracy2AD).replace(p,c) + ' %')
#print ('Random Forest                          :  ' + repr(accuracy2RF).replace(p,c) + ' %')
#print ('Redes Neuronales (Pybrain)             :  ' + repr(100 - errorRN).replace(p,c) + ' %')
print ()
print ('_______________________')
print ()
print ('PORCENTAJES DE ACIERTO3 (Diferencia con Error2 - con acercamiento):')
print ()
print ('KNN con K vecinos más cercanos         :  ' + repr(accuracy3KNN).replace(p,c) + ' %')
#print ('SVM con Kernel polinomial Grado 4      :  ' + repr(accuracy3POL).replace(p,c) + ' %')
print ('SVM con Kernel RBF                     :  ' + repr(accuracy3RBF).replace(p,c) + ' %')
#print ('SVM - SVC No Lineal                    :  ' + repr(accuracy3NL).replace(p,c) + ' %')
#print ('SVM - SVC Lineal                       :  ' + repr(accuracy3LN).replace(p,c) + ' %')
#print ('SVM - SVC Naïve Bayes                  :  ' + repr(accuracy3NB).replace(p,c) + ' %')
print ('Árboles de Decisión                    :  ' + repr(accuracy3AD).replace(p,c) + ' %')
print ('Random Forest                          :  ' + repr(accuracy3RF).replace(p,c) + ' %')
print ()
#print ('_______________________')
#print ()
#print ('PORCENTAJES DE ERROR:')
#print ()
#print ('KNN con K vecinos más cercanos (K = 15):  ' + repr(errorKNN).replace(p,c) + ' %')
#print ('SVM con Kernel polinomial Grado 4      :  ' + repr(errorPOL).replace(p,c) + ' %')
#print ('SVM con Kernel RBF                     :  ' + repr(errorRBF).replace(p,c) + ' %')
#print ('SVM - SVC No Lineal                    :  ' + repr(errorNL).replace(p,c) + ' %')
#print ('SVM - SVC Lineal                       :  ' + repr(errorLN).replace(p,c) + ' %')
#print ('SVM - SVC Naïve Bayes                  :  ' + repr(errorNB).replace(p,c) + ' %')
#print ('Árboles de Decisión                    :  ' + repr(errorAD).replace(p,c) + ' %')
#print ('Random Forest                          :  ' + repr(errorRF).replace(p,c) + ' %')
#print ('Redes Neuronales (Pybrain)             :  ' + repr(errorRN).replace(p,c) + ' %')
print ()
print ('_______________________')
print ()
print ('PORCENTAJES DE ERROR2 (con acercamiento):')
print ()
print ('KNN con K vecinos más cercanos         :  ' + repr(error2KNN).replace(p,c) + ' %')
#print ('SVM con Kernel polinomial Grado 4      :  ' + repr(error2POL).replace(p,c) + ' %')
print ('SVM con Kernel RBF                     :  ' + repr(error2RBF).replace(p,c) + ' %')
#print ('SVM - SVC No Lineal                    :  ' + repr(error2NL).replace(p,c) + ' %')
#print ('SVM - SVC Lineal                       :  ' + repr(error2LN).replace(p,c) + ' %')
#print ('SVM - SVC Naïve Bayes                  :  ' + repr(error2NB).replace(p,c) + ' %')
print ('Árboles de Decisión                    :  ' + repr(error2AD).replace(p,c) + ' %')
print ('Random Forest                          :  ' + repr(error2RF).replace(p,c) + ' %')
#print ('Redes Neuronales (Pybrain)             :  ' + repr(errorRN).replace(p,c))
print ()
print ('_______________________')
print ()

_______________________

Nombre del Departamento: OBSTETRICS CON PATRÓN
_______________________

NÚMERO DE CASOS PARA ENTRENAMIENTO (67%) Y PARA PRUEBAS (33%):

   Entrenamiento: 218
   Pruebas      : 103

_______________________

PORCENTAJES DE ACIERTO EXACTOS:

KNN con K vecinos más cercanos         :  29,126213592233007 %
SVM con Kernel RBF                     :  31,06796116504854 %
Árboles de Decisión                    :  40,77669902912621 %
Random Forest                          :  40,77669902912621 %


_______________________

PORCENTAJES DE ACIERTO3 (Diferencia con Error2 - con acercamiento):

KNN con K vecinos más cercanos         :  62,384805054707947 %
SVM con Kernel RBF                     :  71,431653567575893 %
Árboles de Decisión                    :  78,378794883649263 %
Random Forest                          :  78,792572044999233 %


_______________________

PORCENTAJES DE ERROR2 (con acercamiento):

KNN con K vecinos más cercanos         :  37,615194945292053 %
SVM co